In [1]:
# fix numpy in colab
import numpy
from IPython.display import clear_output
import sys

# fix triton in colab
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

%pip install torch==2.1.0 transformers==4.36.1 hivemind==1.1.9 --quiet
%pip install git+https://github.com/mobiusml/hqq.git@37502bea31f2969c6680c0c4a88ca74b3bb234a5 --quiet
%pip install numpy==1.23.5 --quiet
!git clone https://github.com/dvmazur/mixtral-offloading.git

sys.path.append("mixtral-offloading")
clear_output()

In [2]:
import torch
from torch.nn import functional as F

from transformers import AutoConfig, AutoTokenizer

from huggingface_hub import snapshot_download
from IPython.display import clear_output
from tqdm.auto import trange

from src.build_model import build_model, OffloadConfig


model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
config = AutoConfig.from_pretrained(model_name)

state_path = snapshot_download("lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo")
clear_output()

device = torch.device("cuda:0")

offload_config = OffloadConfig(
    main_size=config.num_local_experts * config.num_hidden_layers * 4 // 8,
    offload_size=config.num_local_experts * config.num_hidden_layers * 4 // 8,
    buffer_size=4,
    offload_per_layer=4,
)

model = build_model(
    device=device,
    offload_config=offload_config,
    state_path=state_path,
)

/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:412: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


  0%|          | 0/32 [00:00<?, ?it/s]

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
id_to_token = {v: k for k, v in tokenizer.get_vocab().items()}


input_ids = tokenizer.apply_chat_template(
    [
      dict(
            role="user",
            content="Integrate exp(-x^2)"
          )
    ],
    return_tensors='pt',
).to(device)

inputs = dict(input_ids=input_ids, attention_mask=torch.ones_like(input_ids))

with torch.no_grad():
    past_key_values = None
    for _ in trange(100):
        out = model(**inputs, past_key_values=past_key_values, output_hidden_states=True)
        past_key_values = out.past_key_values
        probs = F.softmax(out.logits[0, -1] / 0.9, dim=-1)
        token_id = torch.multinomial(probs, 1)
        token = id_to_token[token_id.item()]

        if token[0] == '▁':
            print(end=' ')
            token = token[1:]
        print(token, end='')

        inp = token_id.reshape(1, 1)
        inputs = dict(
            input_ids=inp,
        )

  0%|          | 0/100 [00:00<?, ?it/s]

 The integral of exp(-x^2) is not an elementary function, which means that it cannot be expressed in terms of more familiar functions like polynomials, exponentials, logarithms, and trigonometric functions. This is a special function, which is often denoted by Erfi(x), the "imaginary error function".<0x0A><0x0A>The definite integral of exp(-x^2) from 0 to x can be expressed as <0x0A><0x0A><0xE2><0x88><0xAB><0xE2><0x82>